In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline



# Blind source separation using NMF

Toy example using signals provided by Daniele Borio


In [ ]:
print(__doc__)

import os as os
import numpy as np
import matplotlib.pyplot as plt
from scipy import io
from scipy.signal import stft, istft, spectrogram
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from numpy import linalg


In [ ]:
plt.rcParams['figure.figsize'] = (12.0, 9.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rc('text', usetex=True)
plt.rc('font', family='times')
plt.rcParams.update({'font.size': 26})

In [ ]:
figPath = './figs/'
fs = 10e6

In [ ]:
dataDir = 'data'
fullDir = os.path.join(dataDir, 'JammerData2.mat')
data = io.loadmat(fullDir)
data = np.squeeze(data['data'])

In [ ]:
def plotSpectrogram(f, t, Sxx, vmin, vmax, save_fig=False,fig_path=None, figName=None):
    
    specdB = 10*np.log10(np.abs(Sxx))    
    cmap = plt.get_cmap('jet')
    plt.pcolormesh(np.fft.fftshift(f/1e6), t*1e3, specdB.T,
                  vmin=vmin, vmax=vmax, cmap=cmap)
    plt.xlabel('Frequency [MHz]')
    plt.ylabel('Time [ms]')
    plt.colorbar().ax.set_ylabel('[dB]')
    plt.tight_layout()
    if save_fig:
        if not os.path.exists(fig_path):
            os.makedirs(fig_path)
        plt.savefig(os.path.join(fig_path,figName), dpi=100, format='eps')
    plt.show()

In [ ]:
from NMF_test import stft_fun
from NMF_estimator import NMF_estimator

In [ ]:
pipe = Pipeline([('spectrogram_eval', stft_fun(fs=fs)),
                 ('separate_sources', NMF_estimator(init='random', max_iter=1000, tol=1e-6, verbose=2, 
                  random_state=0, solver='mu'))])
    
param_grid = {'spectrogram_eval__nperseg': [2**8, 2**9, 2**10], 
              'spectrogram_eval__nfft': [2**10, 2**11, 2**12], 
              'separate_sources__n_components': [2], 'separate_sources__beta_loss':
             ['frobenius','kullback-leibler']}

grid = GridSearchCV(pipe, cv=[(slice(None), slice(None))], n_jobs=-1, param_grid=param_grid)
grid.fit(data)

In [ ]:
grid.best_params_

In [ ]:
NMFAttr = grid.best_estimator_.named_steps['separate_sources']
H = NMFAttr.components_
W = NMFAttr.W_
numberOfSources = W.shape[1]

In [ ]:
100000/512


In [ ]:
dataHat = grid.best_estimator_.named_steps['spectrogram_eval'].inverse_transform(W, H)

In [ ]:
grid.best_score_ 

In [ ]:
Zxx = stft_fun(fs=fs, nperseg=grid.best_params_['spectrogram_eval__nperseg'],
               nfft=grid.best_params_['spectrogram_eval__nfft'])

Zxx2 = Zxx.transform(data)
phase = np.angle(Zxx.Zxx)

dataHat2 = []
for i in range(numberOfSources):
    x = np.outer(W[:,i],H[i,:])
    x = x*np.exp(1j*phase)
    _, y = istft(x, fs=fs, nperseg=grid.best_params_['spectrogram_eval__nperseg'],
                 nfft=grid.best_params_['spectrogram_eval__nfft'], input_onesided=False)
    dataHat2.append(y)
print(np.linalg.norm(dataHat[0]-dataHat2[0]))
print(np.linalg.norm(dataHat[1]-dataHat2[1]))

In [ ]:
print(W.shape,H.shape)

In [ ]:
NMFSep = {}
NMFSep['data'] = dataHat
io.savemat('data/NMFbest2.mat',NMFSep)

In [ ]:
f, t, Sxx = spectrogram(data, fs=fs, nperseg=grid.best_params_['spectrogram_eval__nperseg'],
                            nfft=grid.best_params_['spectrogram_eval__nfft'])
specdB = 20*np.log10(np.abs(Sxx))   
cmin = np.min(specdB)
cmax = np.max(specdB)
print(cmin,cmax)
plotSpectrogram(f, t, Sxx, cmin, cmax, save_fig=True, fig_path=figPath, figName=''.join(['JammerData','.eps']))

In [ ]:
aux = 'NMFData'
for j, i in enumerate(dataHat):
    f, t, Sxx = spectrogram(i, fs=fs, nperseg=grid.best_params_['spectrogram_eval__nperseg'],
                            nfft=grid.best_params_['spectrogram_eval__nfft'])
    plotSpectrogram(f, t, Sxx, cmin, cmax, save_fig=True, fig_path=figPath, figName=''.join([aux,str(j),'.eps']))

# Spectrogram of the reconstructed signal

In [ ]:
np.dot(W,H).shape
# print(Sxx.shape) fix it

In [ ]:
# plotSpectrogram(f, t, np.dot(W,H), cmin, cmax, save_fig=True, fig_path=figPath, 
#                 figName=''.join(['ReconstSignal','.eps']))

# Comparing to the adaptive notch filtering result

In [ ]:
fullDir = os.path.join(dataDir, 'filteredData.mat')
filteredData = io.loadmat(fullDir)
filteredData = np.squeeze(filteredData['data'])

In [ ]:
f, t, Sxx = spectrogram(filteredData, fs=fs, nperseg=grid.best_params_['spectrogram_eval__nperseg'],
                            nfft=grid.best_params_['spectrogram_eval__nfft'])
plotSpectrogram(f, t, Sxx, cmin, cmax, save_fig=True, fig_path=figPath, figName=''.join(['notch','.eps']))

# Spectrogram of pure satellite signal

In [ ]:
fullDir = os.path.join(dataDir, 'satSignal.mat')
satSignal = io.loadmat(fullDir)
satSignal = np.squeeze(satSignal['locC'])

In [ ]:
f, t, Sxx = spectrogram(satSignal, fs=fs, nperseg=grid.best_params_['spectrogram_eval__nperseg'],
                            nfft=grid.best_params_['spectrogram_eval__nfft'])
plotSpectrogram(f, t, Sxx, cmin, cmax)

In [ ]:
f, t, Sxx = spectrogram(data, fs=fs, nperseg=grid.best_params_['spectrogram_eval__nperseg'],
                            nfft=grid.best_params_['spectrogram_eval__nfft'])

print(Sxx.shape,data.shape)
print(Sxx.shape[0]*Sxx.shape[1])
print(dataHat[0].shape)